In [ ]:
!pip install langchain langchain_community jq
!pip install faiss-gpu

Database

In [ ]:
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time
import os


In [ ]:

json_file = "articles.json"
topic = "Health"


loader = JSONLoader(
    file_path=json_file,
    jq_schema=f'.{topic}[]',
    content_key="summary"
)

documents = loader.load()

print(f"Loaded {len(documents)} documents for topic: {topic}")


In [ ]:
# Break documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
split_documents = []
for doc in tqdm(documents, desc="Chunking documents"):
    chunks = text_splitter.split_text(doc.page_content)
    split_documents.extend([
        Document(page_content=chunk, metadata=doc.metadata) for chunk in chunks
    ])

print(f"Generated {len(split_documents)} chunks from the documents.")


In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")
def compute_embeddings_parallel(documents, embedding_model):
    embeddings = []
    with ThreadPoolExecutor() as executor:
        for embedding in tqdm(
            executor.map(embedding_model.embed_query, [doc.page_content for doc in documents]),
            total=len(documents),
            desc="Processing documents"
        ):
            embeddings.append(embedding)
    return embeddings

print("Creating FAISS vector database for {} documents...".format(len(split_documents)))
embeddings = compute_embeddings_parallel(split_documents, embedding_model)
vectorstore = FAISS.from_documents(split_documents, embedding_model)
print("FAISS vector database created successfully.")
vectorstore.save_local("/content/drive/MyDrive/faiss_index")
print("FAISS index saved to disk.")


In [ ]:
# Load FAISS index from disk (if already saved)
vectorstore = FAISS.load_local("/content/drive/MyDrive/faiss_index", embedding_model, allow_dangerous_deserialization = True)
print("Loaded FAISS index from disk.")
# User query
user_query = input("Enter your question: ")

# Retrieve the most relevant context using Maximum Marginal Relevance
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3, "fetch_k": 6})
results = retriever.get_relevant_documents(user_query)  # Correct method

# Extract context
if results:
    context = results[0].page_content
    print("\nRetrieved Context:")
    print(context)
else:
    print("\nNo relevant context found.")



-------------------------------------------------------

llama 

In [ ]:
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time
import os


In [ ]:

model_name = "llama2"
llama_model = Ollama(model=model_name)

explicit_prompt_template = """
You are an AI model that provides detailed and specific answers. Answer the question based on the context below. If you cannot answer the question, reply "I don't know". Limit your response to 30 words.

Context: {context}

Question: {question}
"""
explicit_prompt = PromptTemplate.from_template(explicit_prompt_template)
hardcoded_context = "to the yellow fever vaccine is known as yellow fever vaccine associated acute neurotropic disease yel and the canadian medical association published a 2001 cmaj article entitled yellow fever"
hardcoded_query = "What is yellow fever vaccine?"

input_data = {"context": hardcoded_context, "question": hardcoded_query}

start_time = time.time()
answer = llama_model.invoke(input=explicit_prompt.format(**input_data))
end_time = time.time()

time_taken = end_time - start_time

print("\nLlama Answer:")
print(answer)
print(f"\nTime Taken for Llama to Answer: {time_taken:.4f} seconds")
